In [68]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
import requests


In [69]:
d1 = date(2016, 1, 1)  # start date
d2 = date(2016, 12, 31)  # end date

delta = d2 - d1  # timedelta

dates = []
for i in range(delta.days + 1):
    dates.append((d1 + timedelta(days=i)).strftime("%Y/%#m/%#d"))

urls = []
for i in dates:
        urls.append("https://www.wunderground.com/history/airport/KJFK/" + i + "/DailyHistory.html")

In [72]:
# Pull Headers and Build Dataframe
headers = []
headR = requests.get(urls[1])
headData = headR.text
headSoup = BeautifulSoup(headData, "lxml")
headTable = headSoup.find('table', { "id" : "obsTable"})
for row in headTable.find_all('th'):
    header = row.renderContents()
    headers.append(header.decode().rstrip())

headers[0] = headers[0].split()[0]
headers[1] = headers[1][:-1]
headers.insert(0, 'Date')

df = pd.DataFrame(columns = headers)

In [74]:
for i in urls:
    r = requests.get(i)
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    table = soup.find('table', { "id" : "obsTable"})
    rows = table.find_all('tr')
    urlDate = datetime.strptime(soup.find('h2', { "class" : "history-date"}).text, "%A, %B %d, %Y").date()
    for j in rows:
        cells = j.find_all("td")
        row = [i.text.strip().replace('\xa0', '').replace('°F','').replace('in','').replace('mi','').replace('mph','').replace('-','') for i in cells]
        if len(row) == 13:
            row.insert(0, urlDate)
            appRow = pd.Series(row, index = headers)
            df = df.append(appRow, ignore_index=True)




In [82]:
# print(df[6450:])

In [81]:
df.to_csv("nyc_weather_2016.csv")